# Interpolation

Interpolate the advected particles of salinity into the insitu data dimensions

In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import dates as mdates
from datetime import datetime
from scipy import interpolate
from os import path
from mpl_toolkits.basemap import Basemap

In [2]:
# inputs
pathSMOS = '../../data/SSS_data/SMOS_L3_DEBIAS_LOCEAN_v8/debiasedSSS_09days_v8/*.nc'
pathTSG = '../../data/insitu_data/Manta2016/TSG.csv'
pathVEL = '../../data/velocity_data/cmems_obs-sl_glo_phy-ssh_my_allsat-l4-duacs-0.25deg_P1D_1701257133382.nc'
mainpathSIM = "../../outputs/Manta2016/" # main path of the simulation
pathSIM = mainpathSIM + "sim/" # path of simulation outputs
pathADVSSS = mainpathSIM + "sim_tagged/" # path to store advected sss
pathSSSInterpolation = mainpathSIM + "sim_interpolation/"
pathSMOSInterpolation = mainpathSIM + "SMOS_interpolation/"

# outputs
pathPLOT = "../../plots/Manta2016/SSS_comparison/" # path for plots

### Parameters

In [3]:
# running days
rds = [8]

# save figures?
save = True 

# domain limits
time_lim = ["2016-04-08", "2016-05-12"]
lonmin, lonmax = -60, -45
latmin, latmax = -45, -30

# SMOS SSS L3 maps generated by CATDS CEC LOCEAN (9 days)
dsSMOS = xr.open_mfdataset(pathSMOS)
dsSMOS = dsSMOS.sel(time=slice(time_lim[0],time_lim[1]),lon=slice(lonmin,lonmax),lat=slice(latmin,latmax))

# TSG
dsTSG = pd.read_csv(pathTSG)

In [4]:
def map_plot(map,lon_lim,lat_lim,ax):
    """Decorations for plotting geopositional data"""
    #map.drawcoastlines(ax=ax)
    #map.drawrivers(ax=ax)
    map.fillcontinents(color='0.8', lake_color='0.8',ax=ax)

    parallels = np.arange(lat_lim[0]+1,lat_lim[1],2)
    meridians = np.arange(lon_lim[0]+1,lon_lim[1],2)
    map.drawparallels(parallels,labels=[1,0,0,0],fontsize=6, linewidth=0.1, ax=ax)
    map.drawmeridians(meridians,labels=[0,0,0,1],fontsize=6, linewidth=0.1, ax=ax)
    ax.spines[:].set_visible(False)

def plot_insitu(ds_TSG,ax,variable,cmap,vmin,vmax):
    """Plot in-situ data. Scatter with cmap of salinity"""
    ax.scatter(ds_TSG["longitude"],ds_TSG["latitude"],s=25,c='w')
    plot = ax.scatter(ds_TSG["longitude"],ds_TSG["latitude"],c=ds_TSG[variable],cmap=cmap,vmin=vmin,vmax=vmax,s=20,label="")
    return plot

map = Basemap(resolution='i',area_thresh=10,llcrnrlon=lonmin,llcrnrlat=latmin,urcrnrlon=lonmax,urcrnrlat=latmax)

In [5]:
# extract SMOS and TSG time dimension
timeSMOS = np.array(dsSMOS.time.values,dtype="datetime64[s]")
timeTSG = [np.array(dsTSG.date.values[i],dtype="datetime64[s]") for i in range(len(dsTSG))]
dsTSG.date = timeTSG

In [6]:
for rd in rds:
     
     pathADVSSS_rd = pathADVSSS + "advection" + '%02i'% rd + "days/" # path of advected sss
     pathSSSInterpolation_rd = pathSSSInterpolation + "advection" + '%02i'% rd + "days/"
     pathSIM_rd = pathSIM + "advection" + '%02i'% rd + "days/*.zarr"
     
     dsSIM = xr.open_mfdataset(pathSIM_rd,engine="zarr")
     
     for date in timeSMOS:

          date_obj = date.item()

          dsSMOS_date = dsSMOS.sel(time=date)

          """Open TSG data for this date"""

          dsTSG_date = dsTSG[(dsTSG.date > date-np.timedelta64(2,"D")) & (dsTSG.date < date+np.timedelta64(2,"D"))]

          """Open data from simulation"""

          dsSIM_date = dsSIM.sel(obs=np.max(dsSIM.obs))
          lon_sim = dsSIM_date.lon
          lat_sim = dsSIM_date.lat 

          """Open tagged SSS"""

          filename = "sim"+ str(date)[:4] + str(date)[5:7] + str(date)[8:10] +"_" + '%02i'% rd + "days_sss.nc"

          print(filename)

          ds_tag = xr.open_dataset(pathADVSSS_rd + filename)
          sss_tag = ds_tag.sss_adv.values
          lon_tag = ds_tag.lon.values
          lat_tag = ds_tag.lat.values
          
          """Interpolate the corresponding SSS from simulation to in-situ measure positions"""

          sss_int = interpolate.griddata((lon_tag.ravel(),lat_tag.ravel()),sss_tag,(dsTSG_date.longitude,dsTSG_date.latitude))

          """Save dataset of advected sss"""

          name_file = pathSSSInterpolation_rd + filename[:18] + "_sss_int_insitu.nc"
          
          if path.exists(name_file) == False:
                    #shutil.copyfile(path_save+filename,name_file)
                    print("")
                    print("file for sss created, ... ", name_file)

                    # Create variable
                    ds = xr.Dataset(
                         data_vars = dict(
                              sss_adv = (["time"],sss_int)
                         ),
                         coords = dict(
                              lon = (["time"],dsTSG_date.longitude),
                              lat = (["time"],dsTSG_date.latitude),
                              time = (dsTSG_date.dates)
                         )
                    )
                    ds.to_netcdf(name_file)
          
          """"""
          name = str(date)[:4] + str(date)[5:7] + str(date)[8:10] +"_SMOS_int_insitu.nc"

          dsSMOSInterpolation = xr.open_dataset(pathSMOSInterpolation + name)

          # ------------------------------------------------------------------------------------------------------------
          
          fig,axs = plt.subplots(2,2,figsize=(12,6),height_ratios=[1,.2])
          fig.suptitle(str(date)[:10])
          gs = axs[1,1].get_gridspec()
          # remove the underlying axes
          for ax in axs[1,:]:
               ax.remove()
          axbig = fig.add_subplot(gs[1,:])
          """Plot the advected field at TF"""

          map_plot(map,[lonmin,lonmax],[latmin,latmax],axs[0,0])
          sc = axs[0,0].scatter(lon_tag,lat_tag,c=sss_tag,vmin=31,vmax=37,cmap=plt.cm.jet,s=.3)
          axs[0,0].set_title("Advected field")
          axs[0,0].set_xlim(-60,-45)
          axs[0,0].set_ylim(-45,-30)

          map_plot(map,[lonmin,lonmax],[latmin,latmax],axs[0,1])
          sc = axs[0,1].pcolor(dsSMOS_date.lon,dsSMOS_date.lat,dsSMOS_date.SSS,vmin=31,vmax=37,cmap=plt.cm.jet)
          axs[0,1].set_title("SMOS L3")
          axs[0,1].set_xlim(-60,-45)
          axs[0,1].set_ylim(-45,-30)

          cbar = fig.colorbar(sc, ax=axs[0], pad=.01)

          plot_insitu(dsTSG_date,axs[0,0],"salinity_psu",plt.cm.jet,31,37)
          plot_insitu(dsTSG_date,axs[0,1],"salinity_psu",plt.cm.jet,31,37)

          axbig.plot(dsTSG_date.dates.values,sss_int,label="advection",color="r")
          axbig.plot(dsTSG_date.dates.values,dsSMOSInterpolation.sss.values,label="SMOS",color="b")
          axbig.scatter(dsTSG_date.dates.values,dsTSG_date.salinity_psu,label="in-situ",s=.8,c="k")

          axbig.set_xticks([dsTSG_date.dates.values[int(i)] for i in np.linspace(0,len(dsTSG_date)-1,5)])
          axbig.set_xticklabels([str(dsTSG_date.date.values[int(i)])[8:16] for i in np.linspace(0,len(dsTSG_date)-1,5)])
          axbig.set_ylabel("(psu)")
          axbig.spines[:].set_visible(False)
          axbig.grid()

          plt.legend(loc='best',fontsize=6,frameon=False)

          plt.savefig("../../plots/Manta2016/SSS_comparison/"+ "advection" + '%02i'% rd + "days/" +filename[3:18]+".png",dpi=300,bbox_inches='tight',pad_inches = 0.1)
          plt.close()


sim20160410_08days_sss.nc

file for sss created, ...  ../../outputs/Manta2016/sim_interpolation/advection08days/sim20160410_08days_sss_int_insitu.nc
sim20160414_08days_sss.nc

file for sss created, ...  ../../outputs/Manta2016/sim_interpolation/advection08days/sim20160414_08days_sss_int_insitu.nc
sim20160418_08days_sss.nc

file for sss created, ...  ../../outputs/Manta2016/sim_interpolation/advection08days/sim20160418_08days_sss_int_insitu.nc
sim20160422_08days_sss.nc

file for sss created, ...  ../../outputs/Manta2016/sim_interpolation/advection08days/sim20160422_08days_sss_int_insitu.nc
sim20160426_08days_sss.nc

file for sss created, ...  ../../outputs/Manta2016/sim_interpolation/advection08days/sim20160426_08days_sss_int_insitu.nc
sim20160430_08days_sss.nc

file for sss created, ...  ../../outputs/Manta2016/sim_interpolation/advection08days/sim20160430_08days_sss_int_insitu.nc
sim20160504_08days_sss.nc

file for sss created, ...  ../../outputs/Manta2016/sim_interpolation/advection0